In [1]:
from sklearn.datasets import load_breast_cancer
cancer = load_breast_cancer()

In [2]:
from sklearn.preprocessing import StandardScaler #표준화. 평균을 0으로, 표준편차를 1로 만들어주는
cancer_std = StandardScaler().fit_transform(cancer.data)

In [3]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    cancer_std, cancer.target, stratify=cancer.target, test_size=0.2, random_state=2023
)

1. Voting 방식

##### 1) Hard Voting
- 로지스틱 회귀
- 서포트 벡터 머신
- K 최근접 이웃

In [5]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

In [6]:
lrc = LogisticRegression(random_state=2023)
svc = SVC(random_state=2023)
knn = KNeighborsClassifier(n_neighbors=5) #이웃의 수=5

In [7]:
from sklearn.ensemble import VotingClassifier

voc = VotingClassifier(
    estimators=[('LRC', lrc), ('SVC', svc), ('KNN', knn)],
    voting='hard'
)

#VotingClassifier: Hard Voting을 사용. 앙상블 분류기 중 하나. 여러 개의 개별 분류기의 예측 결과를 조합하여 최종 예측 수행
#estimators: 보팅 분류기에 사용할 개별 분류기. 각 분류기는 튜플형태('분류기 이름', 분류기 객체)로 지정

In [9]:
voc.fit(X_train, y_train)
voc.score(X_test, y_test)

0.9298245614035088

In [10]:
#개별 분류기의 학습과 평가
lrc.fit(X_train, y_train)
svc.fit(X_train, y_train)
knn.fit(X_train, y_train)
lrc.score(X_test, y_test), svc.score(X_test, y_test), knn.score(X_test, y_test)

(0.9473684210526315, 0.9298245614035088, 0.9122807017543859)

##### 2) Soft Voting
- predict_proba() 메소드를 지원하는 분류기만 가능
- 로지스틱 회귀
- 서포트 벡터 머신
- K 최근접 이웃

In [11]:
svc2 = SVC(probability=True, random_state=2023)
svc2.fit(X_train, y_train)
svc2.predict_proba(X_test[:3])

#probability=True로 설정된 svc2 모델을 사용해 처음 3개 샘플에 대한 클래스별 예측 확률을 얻을 수 있음
#SVC 모델에서 클래스별 예측 확률을 얻으려면 probability=True 설정

array([[9.99574375e-01, 4.25625266e-04],
       [5.14249474e-08, 9.99999949e-01],
       [1.65822655e-02, 9.83417734e-01]])

In [12]:
voc2 = VotingClassifier(
    estimators=[('LRC', lrc), ('SVC', svc2), ('KNN', knn)], #('분류기 이름', 분류기 객체)
    voting='soft'
)
voc2.fit(X_train, y_train)
voc2.score(X_test, y_test)

#하드 보팅: 클래스 단위의 '다수결'로 최종 예측. 예측 결과 중 가장 많이 선택된 클래스를 최종 예측 클래스로 설정
#소프트 보팅: '평균' 또는 가중 평균하여 최종 예측. 일반적으로 하드 보팅보다 성능이 우수

0.9298245614035088

- GridSearchCV 적용

In [13]:
lrc.C, svc.C

#GridSearch: 하이퍼파라미터(=학습 과정에 영향을 미치는 매개변수. 사용자가 직접 지정)의 최적 값을 찾기 위한 탐색 방법
#lrc.C: 로지스틱 회귀 분류기의 정규화 파라미터(C)에 접근
#svc.C: SVM 분류기의 정규화 파라미터(C)에 접근
#적절히 설정하여 모델의 성능 최적화. C값이 클수록 훈련 데이터에 더 많이 적합 BUT 과적합의 위험 증가

(1.0, 1.0)

In [14]:
from sklearn.model_selection import GridSearchCV #그리드 서치 객체 생성. 최적의 하이퍼파라미터 조합을 찾음
params = {
    'LRC__C': [0.1, 1, 10],
    'SVC__C': [0.1, 1, 10]
}
grid_voc2 = GridSearchCV(voc2, params, scoring='accuracy', cv=5)
grid_voc2.fit(X_train, y_train)
grid_voc2.best_params_ #최적의 하이퍼파라미터 조합을 반환

#그리드 서치를 사용하여 보팅 분류기인 voc2의 하이퍼파라미터를 튜닝
#params: 탐색할 하이퍼파라미터 그리드 정의
#scoring='accuracy': 모델의 성능을 평가할 때 사용할 지표='정확도'
#cv=5: 교차 검증의 폴드 수

{'LRC__C': 10, 'SVC__C': 1}

In [15]:
params = {
    'LRC__C': [5, 10, 20],
    'SVC__C': [0.5, 1, 3]
}
grid_voc2 = GridSearchCV(voc2, params, scoring='accuracy', cv=5)
grid_voc2.fit(X_train, y_train)
grid_voc2.best_params_

{'LRC__C': 10, 'SVC__C': 1}

In [16]:
grid_voc2.best_estimator_.score(X_test, y_test)

#grid_voc2.best_estimator_: 그리드 서치를 통해 찾은 최적의 보팅 분류기 모델
#score(X_test, y_test): 입력 데이터(X_test)에 대한 예측을 수행하고, 이를 실제 레이블(y_test)과 비교하여 정확도 계산

0.9473684210526315

2. Bagging 방식 - Random Forest

In [17]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(random_state=2023)
rfc.get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'sqrt',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'monotonic_cst': None,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': 2023,
 'verbose': 0,
 'warm_start': False}

In [18]:
rfc.fit(X_train, y_train)
rfc.score(X_test, y_test)

0.9210526315789473

In [19]:
rfc.predict_proba(X_test[:5])

#랜덤 포레스트 분류기 모델을 사용하여 테스트 데이터셋의 처음 다섯개 샘플에 대한 클래스별 확률을 예측

array([[0.99, 0.01],
       [0.  , 1.  ],
       [0.18, 0.82],
       [0.  , 1.  ],
       [0.02, 0.98]])

In [ ]:
from sklearn.tree import DecisionTreeClassifier
dtc = DecisionTreeClassifier()
dtc.fit(X_train, y_train)
dtc.predict_proba(X_test[:5])

#DecisionTreeClassifier는 Soft Voting을 하는데 도움이 안됨
#각 노드에서 특성의 임계값을 기준으로 데이터를 분할하여 클래스를 예측 => 클래스별 확률이 아닌, 각 샘플에 대한 클래스 레이블을 직접 예측
#predict_proba 메소드는 DecisionTreeClassifier에서는 사용되지 않음